In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge
from collections import Counter
import Outils
#pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

# A FAIRE : 1
- "triangle" multiple : TRONROUT0000000032823937
- mauvais split : TRONROUT0000000032848625
- gestion separation
- 2*2 voies voies communales : TRONROUT0000000032848788 (ss les rd points ç amarche bien, ça pourrait etre une solution...)

# IMPORTER LES DONNEES 
> depuis la base de données locale, en ne prenant en compte que les lignes qui intersectent une zone carré au N-O

In [196]:
df=at.import_donnes_base('local_otv')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]

# 1 RONDS POINTS

In [197]:
df_avec_rd_pt,carac_rd_pt=at.identifier_rd_pt(df)

In [186]:
df_avec_rd_pt[['id_ign', 'geom', 'id', 'nature', 'numero', 'importance', 'cl_admin',
       'gestion', 'mise_serv', 'fictif', 'franchisst', 'largeur', 'z_ini',
       'z_fin', 'nb_voies', 'sens', 'etat', 'long_km', 'coment_cpt', 'src_cpt',
       'id_comptag', 'obs_supl', 'ann_pt', 'coment_tmj', 'tmja', 'tmja_final',
       'pc_pl', 'obs_tmja', 'obs_pc_pl', 'type_vdf', 'vts_type', 'vma',
       'veh_km', 'id_cpt1', 'id_cpt2', 'obs_tmj1', 'obs_tmj2', 'id_sect',
       'src_sect', 'autor_sect', 'codau_cat', 'milieu', 'vts_vl_vdf',
       'vts_pl_vdf', 'vts_gest', 'obs_vts', 'vts_osm', 'vts_modif', 'vts_vl_f',
       'vts_pl_f', 'codau', 'codau_cont', 'tmja_cont', 'pc_pl_cont', 'source',
       'target', 'recup', 'attr_modif', 'codevoie_d', 'id_tronc_elem',
       'nb_intrsct_src', 'src_geom', 'nb_intrsct_tgt', 'tgt_geom', 'id_rdpt',
       'nb_rte_rdpt', 'nb_obj_sig_entrant']].to_file(r'R:\OTV\test_linearisation\df_avec_rd_pt.shp')

# 2. TRAITEMENTS 
> La base c'est : 
- regrouper les lignes qui se suivent
- trouver les lignes d'arret
- trouver les lignes suivantes
- voir le cas des lignes suivantes : 
    - rd point
    - mm route qui se sépare
    - rien

><br> L'idée c'est ça  : on met la premiere ligne a tester dans une liste, on l'utilise et on stocke les lignes associées dans une autre liste. ensuite on check les lignes suivantes : si elles sont dans un cas où il faut continuer les traitements, on remplace la ligne de départ dans la liste de départ par ces nouvelles lignes, pour lesquelles on applique de nouveau la recherche des lignes qui se suivent. tant que la liste des lignes suivantes est pa vide on reste dans une boucle while

## 2.1 Troncons de base

In [177]:
# ligne a tester : TRONROUT0000000339411609 TRONROUT0000000032869130 TRONROUT0000000032859410
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
id_lignes='TRONROUT0000000342862858'
#trouver tous les troncons proches qui se suivent
#list_troncon=[id_ligne]+[a for a in at.liste_troncon_base(id_ligne,df_lignes)]
list_troncon=at.liste_complete_tronc_base(id_lignes,df_lignes,[])

## 2.2 lignes d'arret

In [178]:
#trouver les troncons de debut et de fin :ceux pour lesquels on a un nombre de ligne sur target source >2
dico_deb_fin=at.deb_fin_liste_tronc_base(df_lignes, list_troncon)

In [179]:
dico_deb_fin

{0: {'id': 'TRONROUT0000000342862858',
  'type': 'source',
  'num_node': 28800,
  'geom_node': 'POINT(379804.1 6571726.49999999)',
  'voie': 'NC',
  'codevoie': '173006945'},
 1: {'id': 'TRONROUT0000000032845688',
  'type': 'target',
  'num_node': 16643,
  'geom_node': 'POINT(379793.4 6571665.39999999)',
  'voie': 'NC',
  'codevoie': '173006945'}}

In [180]:
ligne='TRONROUT0000000342862858'
for v in dico_deb_fin.values() : 
    if v['id']==ligne :
        voie=v['voie']
        noeud=v['num_node']
        geom_noeud=v['geom_node']
        type_noeud=v['type']
        codevoie=v['codevoie']
        #break
    else : 
        continue

## 2.3 lignes suivantes

In [181]:
#vérifier si une ligne touche un rd point
# aprtir d'une ligne et d'un numero de noeud, on récupère les lignes qui touchent
lignes_adj=df_lignes.loc[((df_lignes['source']==noeud)|(df_lignes['target']==noeud))&(df_lignes.index!=ligne)]

In [182]:
lignes_adj

,geom,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,largeur,z_ini,z_fin,nb_voies,sens,etat,long_km,coment_cpt,src_cpt,id_comptag,obs_supl,ann_pt,coment_tmj,tmja,tmja_final,...,vts_osm,vts_modif,vts_vl_f,vts_pl_f,codau,codau_cont,tmja_cont,pc_pl_cont,source,target,recup,attr_modif,codevoie_d,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt_entrant,codevoie_rdpt_entrant,nb_obj_sig_entrant,valeur_sens,id_ign_entrant
id_ign,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TRONROUT0000000032845656,"(LINESTRING (379804.0999999996 6571726.499999993, 379805.5999999997 6571722.099999992, 379808.6999999997 6571716.299999991, 379811.5999999996 6571713.299999991, 379814.1999999997 6571710.899999994...",19851,Route à 1 chaussée,NC,3,Autre,NC,NR,Non,NC,5.5,4.4,4.5,2.0,Direct,NR,0.044772,estimation,None,None,None,None,/2,6720.0,3360.0,...,None,None,46.0,40.0,111+,None,None,None,28800,16642,None,None,17300#03I,255.0,3,POINT(379804.1 6571726.49999999),4,POINT(379838.5 6571704.29999999),1751.0,3.0,"(NC,)","(173006945, 173006035, 173003140)",6.0,"(Direct, Inverse)","(TRONROUT0000000032845698, TRONROUT0000000032845708, TRONROUT0000000032845694, TRONROUT0000000032845700, TRONROUT0000000032845684, TRONROUT0000000342862858)"
TRONROUT0000000032845699,"(LINESTRING (379805.5999999996 6571741.799999991, 379804.5999999997 6571737.599999993, 379803.9999999997 6571733.499999993, 379804.0999999996 6571726.499999993))",19892,Route à 1 chaussée,NC,3,Autre,NC,NR,Non,NC,5.5,4.4,4.4,2.0,Direct,NR,0.015462,estimation,None,None,None,None,/2,6720.0,3360.0,...,None,None,46.0,40.0,111+,None,None,None,28799,28800,None,None,17300#03I,255.0,2,POINT(379805.6 6571741.79999999),3,POINT(379804.1 6571726.49999999),1751.0,3.0,"(NC,)","(173006945, 173006035, 173003140)",6.0,"(Direct, Inverse)","(TRONROUT0000000032845698, TRONROUT0000000032845708, TRONROUT0000000032845694, TRONROUT0000000032845700, TRONROUT0000000032845684, TRONROUT0000000342862858)"


## 2.4 cas particulier 

### 2.4.1 rd pt

In [47]:
#verfier si une ligne rouche un rd point et recup le numero si c'est le cas
check_rdpt, num_rdpt=at.verif_touche_rdpt(lignes_adj)
if check_rdpt : 
    lignes_rdpt, lignes_sortantes=at.recup_lignes_rdpt(carac_rd_pt,num_rdpt,list_troncon,voie,codevoie)

### 2.4.2 mm route qui se sépare

In [772]:
#comparer le nom de voie de la ligne issue de debut_fin avec celui des lignes adjacentes
if len(set(lignes_adj.source.tolist()+lignes_adj.target.tolist())) : #cas d'une ligne qui separe pour se reconnecter ensuite
    print(lignes_adj.index.tolist())
if (voie==lignes_adj.numero).all() and voie!='NC' : 
    print('if')
    ligne_mm_voie=lignes_adj.index.tolist()
    list_troncon+=ligne_mm_voie
    lignes_suivantes_mm_nom=ligne_mm_voie #on recommence les traitements avec les lignes prises
elif voie=='NC' and codevoie!='NR' and (codevoie==lignes_adj.codevoie_d).all(): 
    print('elif')
    ligne_mm_voie=lignes_adj.index.tolist()
    list_troncon+=ligne_mm_voie
    lignes_suivantes_mm_nom=ligne_mm_voie
else : 
    lignes_adj_tot=lignes_adj.copy()
    lignes_adj_tot['tronc_supp']=lignes_adj_tot.apply(lambda x : at.liste_complete_tronc_base(x.name,df_lignes,[id_lignes]), axis=1)
    geom_l1, lg_l1=at.fusion_ligne_calc_lg(df_lignes.loc[lignes_adj_tot.iloc[0].tronc_supp])
    geom_l2, lg_l2=at.fusion_ligne_calc_lg(df_lignes.loc[lignes_adj_tot.iloc[1].tronc_supp])
    tt_tronc=[x for y in lignes_adj_tot.tronc_supp.tolist() for x in y ]
    tt_noeud=Counter(df_lignes.loc[tt_tronc].source.tolist()+df_lignes.loc[tt_tronc].target.tolist())
    if (min(lg_l1, lg_l2) / max(lg_l1, lg_l2)) > 0.66 : 
        noeuds_fin=[k for k,v in tt_noeud.items() if v==1]
        lignes_jointure=df_lignes.loc[(df_lignes.source.isin(noeuds_fin)) & (df_lignes.target.isin(noeuds_fin))]
        print('dehors')
        if not lignes_jointure.empty : 
            print('dedans')
            lgn_cote_1, lgn_cote_2, angle_1,angle_2,angle_3=at.angle_3_lignes('TRONROUT0000000032842584', lignes_adj, noeud,geom_noeud, type_noeud, df_lignes)
            if angle_3<60 :
                print(tt_tronc)
            else : 
                print('pas angle')
        else : 
            print('exit')
    else : 
        print('pb lg')

['TRONROUT0000000032842596', 'TRONROUT0000000032842599']
dehors
exit


### 2.4.3 trouver les routes coinces entre 2 voies qui se sépare -<|
 ce sont des lignes qui  dont : 
- si une des lignes adjacente à le mm nom de voie
- on prend l'autre si elle a un sens direct ou inverse
- on recherche son source ou target different du num_node de dico_deb_fin
- on cherche les lignes qui touche ce noeud, qui ont le mme nom de voie et un sens direct ou inverse
- on recupere cette ligne, et le num de noeud differnts de celui de la 1ere ligne qui bifurque
- si ce num de noeud correspond à la ligne adj source ou target, alors cette ligne adj source ou target doit être ajoutée à la liste

In [923]:
ligne_depart='TRONROUT0000000318255826'
if voie != 'NC' : 
    lgn_cote_1=lignes_adj.loc[(lignes_adj['numero']!=voie)]  
    if lgn_cote_1.empty : 
        print([]) 
    noeud_centre = lgn_cote_1['source'].values[0] if lgn_cote_1['source'].values[0] != noeud else lgn_cote_1['target'].values[0]
    lgn_suiv=lignes_adj.loc[~lignes_adj.index.isin(lgn_cote_1.index.tolist())]
    if lgn_suiv.empty : 
        print([]) 
    noeud_suiv=lgn_suiv['source'].values[0] if lgn_suiv['source'].values[0] != noeud else lgn_suiv['target'].values[0]

    #dans le cas ou plusieurs ligne a la suite forme le cote du triangle que l'on veut récupérer on veut connaitre le nb de ligne qui touchent le noeud_suiv
    if lgn_suiv['source'].values[0] != noeud : 
        noeud_suiv=lgn_suiv['source'].values[0]
        nb_intersect_noeud_suiv=df_lignes.loc[lgn_suiv.index.values[0]].nb_intrsct_src
    else : 
        noeud_suiv=lgn_suiv['target'].values[0]
        nb_intersect_noeud_suiv=df_lignes.loc[lgn_suiv.index.values[0]].nb_intrsct_tgt

    if  nb_intersect_noeud_suiv==2 : 
        lgn_suiv=df_lignes.loc[at.liste_complete_tronc_base(lgn_suiv.index.values[0],df_lignes,[])] 
        noeud_ss_suite=Counter(lgn_suiv.source.tolist()+lgn_suiv.target.tolist())
        noeud_suiv=[k for k,v in noeud_ss_suite.items() if v==1 and v!=noeud][0]

    id_rattache=lgn_suiv.index.tolist()
    if noeud_centre==noeud_suiv : #ça veut dire une seule et mm lign qui fait 2 cote du triangle, dc on peu de suite garder la ligne suiv car 1 seule route non coupe l'intersec
        print (id_rattache)
    else : # on cherche si une ligne touche ces 2 noeuds, a le mme nom de voie que la ligne de cote  et un sens direct ou inverse
        ligne_a_rattacher=df_lignes.loc[((df_lignes['source']==noeud_centre) | (df_lignes['target']==noeud_centre)) & 
                  (df_lignes['numero']==lgn_cote_1['numero'].values[0]) & (~df_lignes.index.isin(lgn_cote_1.index.tolist())) & 
                  ((df_lignes['source']==noeud_suiv) | (df_lignes['target']==noeud_suiv))]
        if not ligne_a_rattacher.empty : 
            print( id_rattache)
        else :
            print([]) 

['TRONROUT0000000032839436', 'TRONROUT0000000318255825']


In [925]:
at.recup_triangle(ligne,voie, codevoie,lignes_adj, noeud,geom_noeud, type_noeud, df_lignes)

['TRONROUT0000000032839436', 'TRONROUT0000000318255825']

## 2.5 route à 2 chaussées
L'idée c'est si la ligne est une route à 2 chaussée : 
- on prend l'ensemble du troncon elementaire
- on agerge les lignes
- on recupere le centre
- on recupere la longueur
- on pourrait chercher à comparer les bouding box

In [69]:
#dans le cas où les longeuers ne coincident pas
#recuperer la ligne proche
ligne_proche, ligne_filtres, longueur_base=at.trouver_chaussees_separee(list_troncon, df_avec_rd_pt)

In [76]:
at.gestion_voie_2_chaussee(list_troncon, df_avec_rd_pt, 'TRONROUT0000000032831888')[0]

['TRONROUT0000000032831889',
 'TRONROUT0000000329876240',
 'TRONROUT0000000329876239',
 'TRONROUT0000000032831831',
 'TRONROUT0000000032831834',
 'TRONROUT0000000032831830']

# Tests de deroulement global

In [123]:
def lignes_troncon_elem(df_avec_rd_pt, ligne) : 
    """
    trouver les lignes qui appartiennent au même troncon elementaire que la ligne de depart
    en entree : 
        df_avec_rd_pt : df des lignes Bdtopo
        ligne : string : id_ign de la ligne a tester
    en sortie :
        liste_troncon_finale : set de string des id_ign des lignes réunies dans le troncon elementaire
    """
    df_lignes2=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
    lignes_a_tester, liste_troncon_finale, list_troncon2=[ligne],[],[]

    while lignes_a_tester :
        for id_lignes2 in lignes_a_tester :
            list_troncon2=at.liste_complete_tronc_base(id_lignes2,df_lignes2,liste_troncon_finale)
            liste_troncon_finale+=list_troncon2
            dico_deb_fin2=at.deb_fin_liste_tronc_base(df_lignes2, list_troncon2)
            for k, v in dico_deb_fin2.items() :
                lignes_adj2=df_lignes2.loc[((df_lignes2['source']==v['num_node'])|
                                          (df_lignes2['target']==v['num_node']))&
                                         (df_lignes2.index!=v['id'])&(~df_lignes2.index.isin(liste_troncon_finale))]
                
                if lignes_adj2.empty : 
                    continue
                check_rdpt2, num_rdpt2=at.verif_touche_rdpt(lignes_adj2)
                # on attaque la liste des cas possible
                #1. Rond point
                if check_rdpt2 : 
                    lignes_rdpt2, lignes_sortantes2=at.recup_lignes_rdpt(carac_rd_pt,num_rdpt2,list_troncon2,v['voie'],v['codevoie'])
                    liste_troncon_finale+=lignes_rdpt2
                    lignes_a_tester+=lignes_sortantes2
                else : #2. route qui se sépare
                    liste_rte_separe=at.recup_route_split(v['id'],v['voie'],v['codevoie'], lignes_adj2,v['num_node'],v['geom_node'],v['type'], df_lignes2)
                    #print(f"{v['id']},liste_rte_separe {liste_rte_separe}")
                    if liste_rte_separe : 
                        lignes_a_tester+=liste_rte_separe 
                        continue
                    liste_triangle=at.recup_triangle(v['id'],v['voie'],v['codevoie'], lignes_adj2, v['num_node'],v['geom_node'],v['type'], df_lignes2)
                    if liste_triangle :
                        #print(f"{v['id']},liste_rte_separe {liste_rte_separe}, liste_triangle {liste_triangle}")
                        liste_troncon_finale+=liste_triangle 
            lignes_a_tester=[x for x in lignes_a_tester if x not in liste_troncon_finale]
    liste_troncon_finale=list(set(liste_troncon_finale))
    return liste_troncon_finale

In [ ]:
lignes_troncon_elem(df_avec_rd_pt,'TRONROUT0000000342862858')

# 3. Boucler sur 'ensembl des lignes
On va faire tourner les traitements sur l'ensemble des lignes, en partant de la liste des id_ign, pour chaque ligne : 
- si la ligne ne remplit pas les critères d'exclusion on continu, sinon on passe à la suivante
- fonction de recherche des lignes associées
- df des association
- filtre de la liste des id_ign de base

In [198]:
# liste des id_ign
liste_id_ign_base=np.array(df_avec_rd_pt.id_ign.tolist())
len(liste_id_ign_base)

107431

In [214]:
liste_id_ign_base=np.array(['TRONROUT0000000032898124'])

In [215]:
dico_fin={}
dico_erreur={}
lignes_traitees=np.array([],dtype='<U24')
for i,l in enumerate(liste_id_ign_base) :
    if len(lignes_traitees)>=len(liste_id_ign_base) : break
    if i%300==0 : 
        print(i, datetime.now(), f'nb lignes traitees : {len(lignes_traitees)}')
    if l in lignes_traitees : 
        continue
    #critère d'exclusion : si la ligne est une bretelle ou si la ligne à un nb de lignes qui intersecte debut et fin >2, ou si c'est un rd pt
    ligne=df_avec_rd_pt.set_index('id_ign').loc[l]
    if ~np.isnan(ligne['id_rdpt']):
        continue
    else : 
        try : 
            liste_ligne=lignes_troncon_elem(df_avec_rd_pt, l) 
            liste_ligne=[x for x in liste_ligne if x not in lignes_traitees] #filtre des lignes deja affectees
            #print(f'liste tronc base : {liste_ligne}')
            if any([x in df2_chaussees.id_ign.tolist() for x in liste_ligne]) :  
                try : 
                    liste_ligne+=at.gestion_voie_2_chaussee(liste_ligne, df_avec_rd_pt, l)[0]
                except at.ParralleleError as Pe:
                    print(Pe)
                    dico_erreur[Pe.id_ign]=Pe.erreur_type
            lignes_traitees=np.unique(np.append(lignes_traitees,liste_ligne))
        except Exception as e : 
            print(l, e)
    for ligne in liste_ligne : 
        dico_fin[ligne]=i
print('fin : ', datetime.now(), f'nb lignes traitees : {len(lignes_traitees)}')    
df_test=pd.DataFrame.from_dict(dico_fin, orient='index').reset_index()
df_test.columns=['id', 'idtronc']

0 2019-09-13 09:48:39.591301 nb lignes traitees : 0
pas de parrallele trouvee pour les troncons ['TRONROUT0000000217441870', 'TRONROUT0000000217441867', 'TRONROUT0000000217441866', 'TRONROUT0000000032898124', 'TRONROUT0000000217441871', 'TRONROUT0000000032898115', 'TRONROUT0000000217441864', 'TRONROUT0000000217441862', 'TRONROUT0000000217441869', 'TRONROUT0000000217441872', 'TRONROUT0000000032898125', 'TRONROUT0000000217441863', 'TRONROUT0000000217441848', 'TRONROUT0000000217441868', 'TRONROUT0000000217441865', 'TRONROUT0000000217441861', 'TRONROUT0000000217441847']
fin :  2019-09-13 09:48:48.075714 nb lignes traitees : 17


In [216]:
df_test

,id,idtronc
0,TRONROUT0000000217441870,0
1,TRONROUT0000000217441867,0
2,TRONROUT0000000217441866,0
3,TRONROUT0000000032898124,0
4,TRONROUT0000000217441871,0
5,TRONROUT0000000032898115,0
6,TRONROUT0000000217441864,0
7,TRONROUT0000000217441862,0
8,TRONROUT0000000217441869,0
9,TRONROUT0000000217441872,0


In [201]:
df_test_export=df_avec_rd_pt.merge(df_test, left_on='id_ign',right_on='id')[['id_ign','numero','codevoie_d','geom','idtronc']]

In [204]:
df_test_export.to_file(r'R:\OTV\test_linearisation\test_global.shp')

In [118]:
df_avec_rd_pt.merge(df_test, left_on='id_ign',right_on='id')[['id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'R:\OTV\test_linearisation\test_9.shp')

In [660]:
# liste des id_ign
liste_id_ign_base=np.array(df_avec_rd_pt.id_ign.tolist())
dico_fin={}
lignes_traitees=np.array([],dtype='<U24')
df_avec_rd_pt2=df_avec_rd_pt.copy()
for i,l in enumerate(liste_id_ign_base) :
    df_avec_rd_pt2=df_avec_rd_pt2.loc[~df_avec_rd_pt2.id_ign.isin(lignes_traitees)]
    if len(lignes_traitees)>=len(liste_id_ign_base) : break
    if l in lignes_traitees : 
        continue
    if i%1000==0 : 
        print(i, datetime.now(), f'nb lignes traitees : {len(lignes_traitees)}')
    #critère d'exclusion : si la ligne est une bretelle ou si la ligne à un nb de lignes qui intersecte debut et fin >2, ou si c'est un rd pt
    ligne=df_avec_rd_pt2.set_index('id_ign').loc[l]
    if ~np.isnan(ligne['id_rdpt']):
        continue
    else :
        liste_ligne=lignes_troncon_elem(df_avec_rd_pt2, l)  
        if l in df2_chaussees.id_ign.tolist() :  
            try : 
                liste_ligne+=at.gestion_voie_2_chaussee(liste_ligne, df_avec_rd_pt2, l)[0]
            except at.ParralleleError :
                pass
        lignes_traitees=np.unique(np.append(lignes_traitees,liste_ligne))
    for ligne in liste_ligne : 
        dico_fin[ligne]=i
print('fin : ', datetime.now(), f'nb lignes traitees : {len(lignes_traitees)}')    
df_test=pd.DataFrame.from_dict(dico_fin, orient='index').reset_index()
df_test.columns=['id', 'idtronc']

0 2019-09-10 17:14:28.657398 nb lignes traitees : 0
2000 2019-09-10 17:54:38.998388 nb lignes traitees : 12507
3000 2019-09-10 18:09:12.964979 nb lignes traitees : 17700
5000 2019-09-10 18:36:33.329216 nb lignes traitees : 26780
8000 2019-09-10 19:11:33.304189 nb lignes traitees : 38800
10000 2019-09-10 19:27:05.616564 nb lignes traitees : 44444
11000 2019-09-10 19:35:23.108986 nb lignes traitees : 47524
13000 2019-09-10 19:51:07.340067 nb lignes traitees : 52527
15000 2019-09-10 20:03:47.255097 nb lignes traitees : 57238
16000 2019-09-10 20:09:33.566982 nb lignes traitees : 59676
17000 2019-09-10 20:15:18.896641 nb lignes traitees : 61947
25000 2019-09-10 20:50:23.285459 nb lignes traitees : 75077
30000 2019-09-10 21:07:24.148735 nb lignes traitees : 81192
34000 2019-09-10 21:18:37.558438 nb lignes traitees : 84955
36000 2019-09-10 21:23:44.598032 nb lignes traitees : 86876
38000 2019-09-10 21:28:22.519167 nb lignes traitees : 88791
42000 2019-09-10 21:37:07.727583 nb lignes traitees 

In [661]:
df_avec_rd_pt.merge(df_test, left_on='id_ign',right_on='id')[['id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'R:\OTV\test_linearisation\test_global.shp')

# SUITE

In [114]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032862365',df,[]))

In [115]:
toto.append('TRONROUT0000000032862365')

In [ ]:
toto

In [117]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000032876496'

In [ ]:
for i in toto :
    tata=at.affecter_troncon_ligne(i,df,df2_chaussees)
    print(len(tata))

In [12]:
dico=at.affecter_troncon(df,df2_chaussees)

0eme occurence : TRONROUT0000000084053027 à 18:23:53 nb ligne traite : 0, nb ligne differente=0
indexError ligne  TRONROUT0000000032954220
indexError ligne  TRONROUT0000000033016282
indexError ligne  TRONROUT0000000249471326
indexError ligne  TRONROUT0000000336640149
indexError ligne  TRONROUT0000000224966440
indexError ligne  TRONROUT0000000032833606
indexError ligne  TRONROUT0000000032895598
indexError ligne  TRONROUT0000000032991823
indexError ligne  TRONROUT0000000223846575
indexError ligne  TRONROUT0000000255628539
indexError ligne  TRONROUT0000000032895522
indexError ligne  TRONROUT0000000032843156
indexError ligne  TRONROUT0000000032966066
indexError ligne  TRONROUT0000000129676068
indexError ligne  TRONROUT0000000032855812
indexError ligne  TRONROUT0000000033029153
indexError ligne  TRONROUT0000000323460798
indexError ligne  TRONROUT0000000032937456
indexError ligne  TRONROUT0000000089679809
indexError ligne  TRONROUT0000000211249489
5000eme occurence : TRONROUT0000000032859682

In [13]:
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [87]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [88]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')
gdf_lignes.to_file(r'D:\temp\otv\17\2_chaussees\lignes_a_gouper.shp')

In [89]:
#union des geometries
gdf_lignes2=linemerge(gdf_lignes.unary_union)

In [ ]:
gdf_lignes2.wkt

In [90]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds
point=gdf_lignes2.interpolate(0.5, normalized=True)

In [91]:
point.wkt

'POINT (385035.1182861876 6561685.156661048)'

In [92]:
#recherche des lignes proches du centroid
lignes_possibles=df.cx[xmin:xmax,ymin:ymax]
lignes_possibles.to_file(r'D:\temp\otv\17\2_chaussees\lignes_possibles.shp')

In [ ]:
lignes_possibles

In [94]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [95]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#garder uniquement la valeur la plus proche du centroid
ligne_proche=ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index.tolist()[0]
ligne_proche

'TRONROUT0000000129676313'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [15]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14] # car un cercle à un rappor de ce type entre 12 et 13
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry']
#on créer aussi la meme donnees avec un buffer interiuer, pour ne garder que les lignes dans le buffer exteriuer et hors buffer interieur (cas de rond point enjmabeant uine 2*2 et prenant la 2*2 voie qui est dans le polygone
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry']

In [63]:
#jointure spataile pour une gdf avec uniquement les lignes des rd_points avec le numéro
l_dans_p=gp.sjoin(df,gdf_rd_point,op='within') 

In [64]:
#suprimer les doublons
l_dans_p_ss_doublon=l_dans_p.reset_index().rename(columns={'index':'id_ign'}).drop_duplicates('id_ign').set_index('id_ign')

In [66]:
l_dans_p_int=gp.sjoin(df,gdf_rd_point_int,op='within')

In [67]:
l_dans_p_final=l_dans_p_ss_doublon.loc[~l_dans_p_ss_doublon.index.isin(l_dans_p_int.index.tolist())]

In [68]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df,l_dans_p_final.drop('index_right', axis=1), how='inner',op='intersects')

In [69]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p_final.index.tolist())][['id_rdpt','numero_left','codevoie_d_left']]

In [70]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x))),
    l_intersct_rdpt.groupby('id_rdpt')['codevoie_d_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt','codevoie_rdpt']

In [ ]:
l_dans_p_final

In [297]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df.drop('id_rdpt',axis=1),l_dans_p_final.loc[:,'id_rdpt']],axis=1, sort=False)

In [298]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [299]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [320]:
def assigne_rdpt(id_rdpt,numero, liste_num, codevoie, liste_code) : 
    if id_rdpt >=0 :
        if (numero in liste_num) or (codevoie in liste_code) :
            return True
        else : 
            return False

In [321]:
gdf_avec_rd_pt['assigne_rdpt']=gdf_avec_rd_pt.apply(lambda x : assigne_rdpt(x['id_rdpt'],x['numero'], x['nom_rte_rdpt_y'], x['codevoie_d'],x['codevoie_rdpt']),axis=1)

In [ ]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [ ]:
#pour export et verif uniquement

#gdf_l_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test2.shp')
#gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
df.to_file(r'E:\Boulot\otv\test_linearisation_auto\test7.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [228]:
id_ign_ligne='TRONROUT0000000342862858'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [ ]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    df_rd_pt=df.loc[df['id_rdpt']==df_touches_source.iloc[0]['id_rdpt']]
    l_df_rd_pt=df.loc[df.index.isin(gp.sjoin(df,df_rd_pt, how='inner',op='intersects').index.unique().tolist())]
    l_df_rd_pt2=l_df_rd_pt.loc[~l_df_rd_pt.index.isin(df_rd_pt.index.tolist())].index.unique().tolist()
    l_df_rd_pt3=df.loc[df.index.isin(l_df_rd_pt2),'codevoie_d'].unique().tolist()
    print(l_df_rd_pt3)
    #liste_ligne_touchees+=df_touches_source.index.tolist()

In [233]:
df_touches_source['id_rdpt'].values.tolist()

[nan]

# TEST

In [138]:
%%time
a=np.array([i for i in range(100000)])
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=a[np.isin(a,c, invert=True)]
    if i%50000==0 : print(i)

AttributeError: module 'numpy' has no attribute 'aray'

In [137]:
%%time
a=[i for i in range(100000)]
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=[x for x in a if x not in c]
    if i%50000==0 : print(i)

0
50000
Wall time: 5min 14s


In [420]:
lignes_traitees=np.array([],dtype='<U24')
'1' in lignes_traitees

False

In [416]:
lignes_traitees.dtype

dtype('<U24')